In [1]:
import logging
import Pyro4

import numpy as np

from bmpga.mating.selectors import BaseSelector 
from bmpga.characterization import SimpleCharacterizer
from bmpga.storage import Database, Cluster, Molecule
from bmpga.systems import DefineSystem
from bmpga.optimisation import PoolGA

In [2]:
log = logging.getLogger(__name__)

compare = SimpleCharacterizer()
database = Database(db="test.db",
                    new_database=True,
                    compare_clusters=SimpleCharacterizer(accuracy=5e-7))

lj = Molecule(coordinates=np.array([[0.0, 0.0, 0.0]]), particle_names=["LJ"])
system = DefineSystem(numbers_of_molecules=[13], molecules=[lj], log=log, box_length=6)

2018-06-10 17:31:31,384 - bmpga.storage.database - WARNING - File test.db exists and new_database is True. This file is being overwritten.
2018-06-10 17:31:31,424 - __main__ - INFO - Initialising RandomClusters with box_length = 6



In [3]:
daemon = Pyro4.Daemon(port=9939)

GA = PoolGA(database=database, 
            min_pool_size=10, 
            max_generations=2,
            system=system, 
            daemon=daemon,
            log=log)

with open("example.uri", "w") as f:
    uri = daemon.register(GA)
    f.write(str(uri))

GA.start_threads()
daemon.requestLoop()
daemon.shutdown()

2018-06-10 17:31:31,777 - __main__ - INFO - Using database: <bmpga.storage.Database @ test.db>

2018-06-10 17:31:31,793 - __main__ - DEBUG - Starting GA thread and result thread
2018-06-10 17:31:31,804 - __main__ - DEBUG - started thread in process_returned
2018-06-10 17:31:31,805 - __main__ - INFO - Beginning global optimisation of <bmpga.systems.define_system.DefineSystem object at 0x7f8474d90da0>. At None
        Using version 0.1 of BMPGA
2018-06-10 17:31:31,806 - __main__ - INFO - Starting the server event loop
2018-06-10 17:31:31,842 - __main__ - DEBUG - Pool not full: 0 < 10. Generating random cluster.
2018-06-10 17:31:31,886 - __main__ - DEBUG - minimum separation < max(molecule_diameter): 0.8731560176047218 < 3
Cluster accepted
2018-06-10 17:31:31,964 - __main__ - INFO - Returning random cluster found after 1 attempts using a final box_length of 6.0.
2018-06-10 17:31:32,023 - __main__ - DEBUG - Pool not full: 0 < 10. Generating random cluster.
2018-06-10 17:31:32,195 - __main_

Exception in thread Thread-8:
Traceback (most recent call last):
  File "/home/john/.local/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/john/.local/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/john/Software/bmpga/bmpga/optimisation/pool_GA.py", line 223, in process_returned
    if self.check_unique_structure(result):
AttributeError: 'PoolGA' object has no attribute 'check_unique_structure'



2018-06-10 17:31:42,864 - __main__ - INFO - Job assigned to worker: 129983989664054829691327285210451767697
2018-06-10 17:31:43,369 - __main__ - INFO - Job assigned to worker: 129983989664054829691327285210451767697
2018-06-10 17:31:43,805 - __main__ - INFO - Job assigned to worker: 129983989664054829691327285210451767697
2018-06-10 17:31:44,150 - __main__ - INFO - Job assigned to worker: 129983989664054829691327285210451767697
2018-06-10 17:31:44,548 - __main__ - INFO - Job assigned to worker: 129983989664054829691327285210451767697
2018-06-10 17:31:44,998 - __main__ - INFO - Job assigned to worker: 129983989664054829691327285210451767697
2018-06-10 17:31:45,399 - __main__ - INFO - Job assigned to worker: 129983989664054829691327285210451767697
2018-06-10 17:31:45,758 - __main__ - INFO - Job assigned to worker: 129983989664054829691327285210451767697
2018-06-10 17:31:46,337 - __main__ - INFO - Job assigned to worker: 129983989664054829691327285210451767697
2018-06-10 17:31:46,939 - __

In [5]:
from bmpga.utils.io_utils import XYZWriter
gm = GA.database.get_global_minimum()
print(gm)
writer = XYZWriter()
writer.write(filename="gm.xyz", structures=gm, file_mode="w")
!jmol gm.xyz

<Cluster (ID=15, Energy=-40.67379777793193, Found at step:5)>
2018-06-10 16:07:58,300 - bmpga.utils.io_utils - WARNING - File gm.xyz exists! This file has been moved to 325328gm.xyz rather than be overwritten with 1 structures.
splash_image=jar:file:/usr/share/java/Jmol-12.2.32.jar!/org/openscience/jmol/app/images/Jmol_splash.jpg
history file is /home/john/.jmol/history
1822 script command tokens
(C) 2012 Jmol Development
Jmol Version: 12.2.32  2012-06-24 02:39
java.vendor: Oracle Corporation
java.version: 1.8.0_171
os.name: Linux
memory: 27.7/124.8
processors available: 4
useCommandThread: false
User macros dir: /home/john/.jmol/macros
       exists: false
  isDirectory: false
Executing script: load "gm.xyz";

FileManager.getAtomSetCollectionFromFile(gm.xyz)
FileManager opening /home/john/Software/bmpga/examples/server_client_notebooks/gm.xyz
The Resolver thinks Xyz
openFile(gm.xyz): 85 ms
reading 13 atoms
ModelSet: haveSymmetry:false haveUnitcells:false haveFractionalCoord:false
1 mo

In [ ]:
GA.write_pool_to_file("pool.xyz")